In [1]:
import pandas as pd
from pymatreader import read_mat
import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from sklearn import preprocessing
import argparse
import time
import matplotlib.pyplot as plt
import traceback
from torch.utils.data import DataLoader, TensorDataset

In [14]:
class args:
    window_size = 600
    step_size = 20
    batch_size = 128
    dilation = 0
    num_layers = 3
    z_score_norm = True
    hidden_size = 12
    input_size = 12
    num_kernels = (32, 64, 64, 128, 128, 256, 256)
    list_dilation = (1, 2, 4, 8, 8, 8, 1)

    learning_rate = 0.001
    factor = 0.5
    patience = 50
    threshold = 1e-2
    lr_limit = 1e-4
    measurement = 'min'
    scheduler = 'plateau'
    optimizer = 'adam'
    weight_decay = 1e-2
    progress_bar = True
    cuda = True
    test = False
    plot = False
    save = False
    scheduler = 'plateau'
    weight_decay = 1e-2

    training_set = (1, 2)
    testing_set = (3, 4)
    epoch = 20
    cuda = cuda and torch.cuda.is_available()
    weight_decay = float(weight_decay)

In [3]:
class Model(nn.Module):
    def __init__(self, args):
        super(Model, self).__init__()
        # single LSTM
        self.window_size = args.window_size
        self.num_layers = args.num_layers
        self.input_size = args.input_size
        self.hidden_size = args.hidden_size
        self.dilation = args.dilation
        self.dilated_n_steps = self.window_size // (self.dilation + 1)
        self.lstm1 = nn.LSTM(input_size=12, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm4 = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, batch_first=True)

        # CNN Blocks
        # num_kernels = (32, 64, 64, 128, 128, 256, 256)
        # list_dilation = (1, 2, 4, 8, 8, 8, 1)
        self.conv1 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=5, dilation=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, dilation=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, dilation=1)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, dilation=1)
        self.conv5 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5, dilation=1)
        self.conv6 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, dilation=1)
        self.conv7 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, dilation=1)
        self.batch_norm1 = nn.BatchNorm1d(num_features=32)
        self.batch_norm2 = nn.BatchNorm1d(num_features=64)
        self.batch_norm3 = nn.BatchNorm1d(num_features=64)
        self.batch_norm4 = nn.BatchNorm1d(num_features=128)
        self.batch_norm5 = nn.BatchNorm1d(num_features=128)
        self.batch_norm6 = nn.BatchNorm1d(num_features=256)
        self.batch_norm7 = nn.BatchNorm1d(num_features=256)
        self.PReLU1 = nn.PReLU()
        self.PReLU2 = nn.PReLU()
        self.PReLU3 = nn.PReLU()
        self.PReLU4 = nn.PReLU()
        self.PReLU5 = nn.PReLU()
        self.PReLU6 = nn.PReLU()
        self.PReLU7 = nn.PReLU()

        # Full connection blocks
        self.Flatten = nn.Flatten(1, 2)
        self.Dense1 = nn.Linear(146432, 64)
        self.Dense2 = nn.Linear(64, 32)
        self.Dense3 = nn.Linear(32, 17)

    def forward(self, x):
#         print(np.shape(x))
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        np.shape(x)
        x = torch.transpose(x, 1, 2)

        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.PReLU1(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.PReLU2(x)
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = self.PReLU3(x)
        x = self.conv4(x)
        x = self.batch_norm4(x)
        x = self.PReLU4(x)
        x = self.conv5(x)
        x = self.batch_norm5(x)
        x = self.PReLU5(x)
        x = self.conv6(x)
        x = self.batch_norm6(x)
        x = self.PReLU6(x)
        x = self.conv7(x)
        x = self.batch_norm7(x)
        x = self.PReLU7(x)
        
#         print('after conv:',x.size())

        x = self.Flatten(x)
#         print('after flat:', x.size())
        x = self.Dense1(x)
#         print('after dense1', x.size())
        x = self.Dense2(x)
        x = self.Dense3(x)
        out = torch.argmax(x, dim=1)

        return out

In [4]:
def data_preprocessing(args, data_set):
    """
    This function is called when loading the data, it takes a lot of time.
    The function is will turn all E1_A1 data to a (x, window_size, 2 + channel_num)
    In which, x is the number of all pieces of windowing,
    window size in paper is set to be 600, which is 300ms
    step_size in paper is set to be 20 which is 10 ms
    channel_num in dataset is 12, fixed.
    Outputs:
        Input: EMG data with 12 channels, shape:(x, window_size, channel_num)
        label: EMG corresponding restimulus digit, shape:(x,)
    """
    data_dir = '/scratch/cw3755/my_project/data_E1/'
    inputs = []
    labels = []
    for i in data_set:
        f = data_dir + '/S' + str(i) + '_E1_A1.mat'
        data_raw = read_mat(f)
        if args.z_score_norm:
            emg = preprocessing.scale(data_raw['emg'])
        df1 = pd.DataFrame(emg)
        df2 = pd.DataFrame(data_raw['restimulus'])
        df3 = pd.DataFrame(data_raw['repetition'])
        df = pd.concat([df3, df2, df1], axis=1)
        for repetition in range(1, 7):
            for restimulus in range(1, 18):
                df4 = df.loc[df.iloc[:, 0] == repetition, :]
                df5 = df4.loc[df.iloc[:, 0] == restimulus, :]
                for step in range((df5.shape[0] - args.window_size) // args.step_size):
                    inputs.append(
                        df5.iloc[args.step_size * step:args.step_size * step + args.window_size, 2:].values)
                    labels.append(restimulus)
    inputs = torch.FloatTensor(np.array(inputs))
    labels = torch.FloatTensor(np.array(labels))
#     print(inputs.size())
    inputs = inputs.cuda()
    labels = labels.cuda()
    inputs = inputs.type(torch.cuda.FloatTensor)
    labels = labels.cuda()
    return inputs, labels

In [5]:
#Creat Net Work
net = Model(args)
if args.cuda:
    net = net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=args.learning_rate, amsgrad=True,
                                   weight_decay=1e-5 * args.batch_size)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, args.measurement, threshold=args.threshold,factor=args.factor,patience=args.patience, min_lr=args.lr_limit)


In [6]:
#load data
print('loading data...')
t1 = time.time()
train_data, train_label = data_preprocessing(args, args.training_set)
test_data, test_label = data_preprocessing(args, args.testing_set)
# print(np.shape(self.train_data))
# print(np.shape(self.train_label))
train = TensorDataset(train_data, train_label)
test = TensorDataset(test_data, test_label)
#train = [[train_data[i, :, :], train_label[i]] for i in range(len(train_label))]
#test = [[test_data[i, :, :], test_label[i]] for i in range(len(test_label))]
# print(np.shape(train))
# print(np.shape(test))

# train_data = train_data.clone().detach().requires_grad_(True)
# test_data = test_data.clone().detach().requires_grad_(True)
# train_label = train_label.clone().detach().requires_grad_(True)
# test_label = test_label.clone().detach().requires_grad_(True)

train_loader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=args.batch_size)
test_loader = torch.utils.data.DataLoader(test, shuffle=False, batch_size=args.batch_size)
t2 = time.time()
print('Data loaded! Time used:', t2 - t1)

loading data...
Data loaded! Time used: 80.67961049079895


In [18]:
train_loss_list = []
test_loss_list = []
test_acc_list = []
lr_list = []

train_dataset = TensorDataset(train_data, train_label)
test_dataset = TensorDataset(test_data, test_label)
trainloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False) 
model = Model(args).cuda()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, amsgrad=True, weight_decay=1e-5 * args.batch_size)
dataset_size = len(trainloader.dataset)
print(dataset_size)

for e in range(1, args.epoch+1):
    train_loss = 0
    test_loss = 0
    test_acc = 0
    if args.progress_bar:
        trainloder = tqdm.tqdm(trainloader)
        testloader = tqdm.tqdm(testloader)
    optimizer.zero_grad()
    for id_batch,(x_batch, y_batch) in enumerate(trainloader):
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        x_batch = x_batch.requires_grad_()
        y_batch = y_batch.requires_grad_()
        y_batch_pred = model(x_batch)
        y_batch_pred = y_batch_pred.type(torch.float32)
        if id_batch == 2:
            print(y_batch_pred)
            print(y_batch)
        loss = loss_fn(y_batch_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if id_batch %10 == 0:
            loss,current = loss.item(), (id_batch+1)*len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")
    if args.test:
        break


    for id_batch,(x_batch, y_batch) in enumerate(testloader):
        with torch.no_grad():
            x_batch = x_batch.cuda()
            y_batch = y_batch.cuda()
            x_batch = x_batch.requires_grad_()
            y_batch = y_batch.requires_grad_()
            y_batch_pred = model(x_batch)
            test_acc += sum(i == j for i,j in zip(y_batch_pred, y_batch))
            loss = loss_fn(y_batch_pred, y_batch)
            test_loss += loss.item()
        if args.test:
            break
    train_loss = train_loss / len(trainloader)
    test_loss = test_loss / len(testloader)
    test_acc = test_acc / 10000
    test_acc = test_acc.tolist()
    lr = optimizer.param_groups[0]['lr']
    
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    test_acc.append(test_acc)
    lr_list.append(lr)
    if args.scheduler == 'plateau':
        if args.measurement == 'min':
            scheduler.step(test_loss)
        else:
            scheduler.step(test_acc)
    print(f'epoch {epoch}, train loss {train_loss:.4}, test loss {test_loss:.4}, test acc {test_acc:.4}, lr {lr:.4}')
if args.save:
    torch.save(model.state_dict(), f"{int(time.time())}.pt")


112378


  0%|          | 0/878 [06:13<?, ?it/s]

  0%|          | 0/874 [00:00<?, ?it/s]

loss: 3674.291748  [  128/112378]
tensor([ 7., 13.,  1., 12.,  0.,  5., 12.,  5.,  5., 13., 12., 13.,  4.,  5.,
        12., 13., 13.,  5., 13.,  3.,  0.,  5.,  9.,  8.,  5.,  5.,  0., 13.,
         5.,  4.,  5., 13., 13.,  5.,  7.,  5., 13.,  0., 13., 12.,  0., 13.,
         7., 13., 13.,  8.,  7.,  5., 10., 13., 13.,  7.,  5.,  5.,  5., 12.,
        10., 12., 10.,  0., 13.,  5.,  5.,  5.,  5., 13.,  5.,  5.,  5.,  0.,
        13., 13., 13., 13.,  3.,  5., 13.,  7., 13., 13., 13.,  5., 13., 13.,
         5.,  5., 12.,  5., 13.,  5., 12., 13., 13., 13.,  5.,  5.,  5.,  0.,
         5.,  5., 13., 13., 13., 13., 13.,  5., 13.,  5.,  5.,  3., 12., 12.,
        12., 13.,  5., 13., 13.,  5., 10.,  5.,  5., 12.,  7.,  5.,  0., 13.,
         5.,  5.], device='cuda:0')
tensor([6., 1., 5., 2., 3., 5., 2., 2., 4., 6., 4., 6., 6., 1., 3., 2., 6., 6.,
        2., 4., 4., 4., 2., 2., 6., 2., 6., 2., 5., 3., 2., 2., 6., 5., 3., 3.,
        6., 4., 3., 4., 3., 5., 6., 4., 4., 2., 2., 4., 5., 3., 5., 

  0%|          | 0/874 [01:07<?, ?it/s]


RuntimeError: "host_softmax" not implemented for 'Long'